This is a simlpe cuGraph example of calculating "Page Rank" for NCAAW basketball teams based on the wins and losses in a single season. We treat two teams as having a "link" between them if they've played a game against each other, with the "link" proceeding from the losing team to the winning team.

I've used cuGraph to calculate Page Rank not becasue I need a lot of speed - we are dealing here with a very small dataset. But primarily becasue I am familiar with it, and it's part of Rapids library, whcih I am all too eager to promote. ;) Unfortunately, cuGraph is still not available in Kaggle Docker, so I have to install it from skrach here from Chris Deotte's dataset.

In [1]:
!nvidia-smi

Sat Sep 11 09:23:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


نصب کوداگراف حدود ۱۵ دقیقه طول می کشد

In [2]:
!conda install -y -c nvidia -c rapidsai -c numba -c conda-forge -c defaults cugraph cudatoolkit=11.0

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit=11.0
    - cugraph


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37h89c1867_1         3.0 MB  conda-forge
    cugraph-21.08.03           |cuda11.0_py37_g9e9f1570_0        62.7 MB  rapidsai
    cupy-9.0.0                 |   py37h4fdb0f7_0        50.3 MB  conda-forge
    dask-cuda-21.08.00         |           py37_0         112 KB  rapidsai
    libcugraph-21.08.03        |cuda11.0_g9e9f1570_0       166.1 MB  rapidsai
    pynvml-11.0.0              |     pyhd8ed1ab_0          39 KB  conda-forge
    ------------------------------------------

In [7]:
# %%time
# import networkx as nx
# import cugraph
# G=nx.erdos_renyi_graph(5000,0.5)
# # b=nx.betweenness_centrality(G)
# bc = cugraph.betweenness_centrality(G)
# bc

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# %cd ..
# !rm -r TC-with-GCN

In [5]:
!git clone https://github.com/mamintoosi/TC-with-GCN.git
%cd TC-with-GCN

Cloning into 'TC-with-GCN'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 124 (delta 69), reused 86 (delta 35), pack-reused 0
Receiving objects: 100% (124/124), 17.81 MiB | 21.97 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/kaggle/working/TC-with-GCN


In [6]:
%%time
%run data_processor.py

mr


clean the text: 10662it [00:00, 26365.76it/s]
10662it [00:00, 26582.22it/s]


Average length: 20.38585631213656
doc count: 10662
Total number of words: 18764
ohsumed


clean the text: 7400it [00:01, 4042.07it/s]
7400it [00:01, 3793.01it/s]


Average length: 135.8177027027027
doc count: 7400
Total number of words: 14157
R52


...

Average length: 69.81989010989011
doc count: 9100
Total number of words: 8892
R8


clean the text: 7674it [00:00, 8280.77it/s]
7674it [00:01, 7554.88it/s]


Average length: 65.72126661454261
doc count: 7674
Total number of words: 7688
20ng


clean the text: 18846it [00:07, 2479.18it/s]
18846it [00:09, 2087.22it/s]


Average length: 221.25697760798047
doc count: 18846
Total number of words: 42757
CPU times: user 26.9 s, sys: 1.16 s, total: 28.1 s
Wall time: 31.4 s


In [7]:
%%time
%run build_graph.py


==> Dataset:mr<==
tfidf time: 0.2816476821899414
tfidf_vec shape: (10662, 18764)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 18764


generate tfidf edge: 10662it [00:02, 4794.11it/s]


+---------+----------+-------------+------------+------------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  Coverage  |
+---------+----------+-------------+------------+------------+
|  29426  |  196826  |      0      |     0      |   1.0000   |
+---------+----------+-------------+------------+------------+
pmi read file len: 10662


Calculate pmi between words: 100%|██████████| 1043553/1043553 [00:02<00:00, 391797.59it/s]


Total number of edges between word: 788993
pmi time: 5.406177043914795
+---------+----------+-------------+------------+------------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  Coverage  |
+---------+----------+-------------+------------+------------+
|  29426  |  917637  |      0      |     0      |   1.0000   |
+---------+----------+-------------+------------+------------+
total time: 5.687824726104736


Computing betweenness centrality...

==> Dataset:ohsumed<==
tfidf time: 0.867239236831665
tfidf_vec shape: (7400, 14157)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 14157


...

Total number of edges between word: 1973932
pmi time: 31.43113398551941
+---------+-----------+-------------+------------+------------+
|  Nodes  |   Edges   |  Selfloops  |  Isolates  |  Coverage  |
+---------+-----------+-------------+------------+------------+
|  17992  |  2101511  |      0      |     0      |   1.0000   |
+---------+-----------+-------------+------------+------------+
total time: 31.999051570892334


Computing betweenness centrality...

==> Dataset:R8<==
tfidf time: 0.46054887771606445
tfidf_vec shape: (7674, 7688)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 7688


generate tfidf edge: 7674it [00:02, 2944.62it/s]


+---------+----------+-------------+------------+------------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  Coverage  |
+---------+----------+-------------+------------+------------+
|  15362  |  323670  |      0      |     0      |   1.0000   |
+---------+----------+-------------+------------+------------+
pmi read file len: 7674


Calculate pmi between words: 100%|██████████| 2327712/2327712 [00:08<00:00, 284827.42it/s]


Total number of edges between word: 1576512
pmi time: 26.4911048412323
+---------+-----------+-------------+------------+------------+
|  Nodes  |   Edges   |  Selfloops  |  Isolates  |  Coverage  |
+---------+-----------+-------------+------------+------------+
|  15362  |  1673139  |      0      |     0      |   1.0000   |
+---------+-----------+-------------+------------+------------+
total time: 26.951653718948364


Computing betweenness centrality...
CPU times: user 16min 46s, sys: 6.98 s, total: 16min 53s
Wall time: 17min 2s


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [9]:
%cd ..
!zip -r -q TCGCN_graphs.zip tmp/

/kaggle/working


In [10]:
!ls

TC-with-GCN  TCGCN_graphs.zip  __notebook_source__.ipynb  tmp


In [11]:
from IPython.display import FileLink
FileLink(r'TCGCN_graphs.zip')

/kaggle/working/TCGCN_graphs.zip

لینک بالا رو در کولب دبلیوگت کردم

In [ ]:
# !cp TCGCN_graphs.zip '/content/gdrive/My Drive/models/'

In [13]:
# from IPython.display import FileLink
# graph_path = "../tmp/TCGCN/graph"
# # dataset = "mr"
# # dataset = "ohsumed"
# # dataset = "R52"
# dataset = "R8"
# file_name = f"{graph_path}/{dataset}_g_info.pkl"
# FileLink(file_name)

In [14]:
# !pip show fastai

In [15]:
# !conda install -y fastai
# !pip install -U fastai

In [16]:
# %%time
# %run trainer.py


 mr
Namespace(data_path='./data', dataset='mr', device=device(type='cuda'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, tmp_path='../tmp/TCGCN', use_gf=False, val_ratio=0.1)
prepare data


ModuleNotFoundError: No module named 'fastai.utils'

CPU times: user 12.4 s, sys: 89.4 ms, total: 12.5 s
Wall time: 12.5 s


In [17]:
!ls

LICENSE     Report.xlsx     data	       requirements.txt  trainer.py
README.md   __pycache__     data_processor.py  run_main.ipynb	 utils.py
Report.txt  build_graph.py  layer.py	       setup.py


In [18]:
# %cd ..
# !git clone https://github.com/mamintoosi/temp_rep.git
# %cd temp_rep
# !cp -r ../tmp ./

/kaggle/working
Cloning into 'temp_rep'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
/kaggle/working/temp_rep


In [1]:
# !ls tmp

ls: cannot access 'tmp': No such file or directory


In [22]:
# !git add *
# !git config user.email "mamintoosi@yahoo.com"
# !git commit -m "Add TCGCN graphs"

[main a6fedd5] Add TCGCN graphs
 13 files changed, 8566931 insertions(+)
 create mode 100644 tmp/TCGCN/clean_corpus/20ng.txt
 create mode 100644 tmp/TCGCN/clean_corpus/R52.txt
 create mode 100644 tmp/TCGCN/clean_corpus/R8.txt
 create mode 100644 tmp/TCGCN/clean_corpus/mr.txt
 create mode 100644 tmp/TCGCN/clean_corpus/ohsumed.txt
 create mode 100644 tmp/TCGCN/graph/R52.txt
 create mode 100644 tmp/TCGCN/graph/R52_g_info.pkl
 create mode 100644 tmp/TCGCN/graph/R8.txt
 create mode 100644 tmp/TCGCN/graph/R8_g_info.pkl
 create mode 100644 tmp/TCGCN/graph/mr.txt
 create mode 100644 tmp/TCGCN/graph/mr_g_info.pkl
 create mode 100644 tmp/TCGCN/graph/ohsumed.txt
 create mode 100644 tmp/TCGCN/graph/ohsumed_g_info.pkl
remote: Support for password authentication was removed on August 13, 2021. Please use a personal access token instead.
remote: Please see https://github.blog/2020-12-15-token-authentication-requirements-for-git-operations/ for more information.
fatal: Authentication failed for 'https